# Sherrington-Kirkpatrick Model

The cost function of the SK model is
$$
\hat H_P = \frac{1}{\sqrt{N}}\sum_{i<j\leq N} J_{ij} \hat{Z}_i \hat{Z}_j,
$$
where the couplings $J_{ij}$ are i.i.d. standard Gaussian variables, i.e. with zero mean $\left\langle J_{ij} \right\rangle = 0$ and variance $ \left\langle J_{ij}^2 \right\rangle = J^2$.

In [ ]:
using QAOA, LinearAlgebra
import Random, Distributions

using PyPlot

## QAOA

__Defining the problem by hand:__

In [ ]:
N = 4
σ2 = 1.0

Random.seed!(1)
J = rand(Distributions.Normal(0, σ2), N, N) ./ sqrt(N) 
J[diagind(J)] .= 0.0
J = UpperTriangular(J)
J = J + transpose(J)

In [ ]:
p = 2
SK_problem = QAOA.Problem(p, zeros(N), J)

__Using the wrapper function:__

In [ ]:
SK_problem = QAOA.sherrington_kirkpatrick(N, σ2, num_layers=p, seed=137)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.02
cost, params, probs = QAOA.optimize_parameters(SK_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(SK_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
xlabels = []
for bstr in digits.(0:2^N-1, base=2, pad=N)
    push!(xlabels, "\$|" * prod([string(b) for b in bstr]) * "\\rangle\$")
end

figure(figsize=(5, 3.2))
ax = subplot(111)
bar(0:2^N-1, probs)
ax.set_xticks(0:2^N-1)
ax.set_xticklabels(xlabels, rotation=90)
minorticks_off()
tight_layout()

## Mean-Field Approximation

In [ ]:
# schedule
p = 1000
τ = 0.5
γ = τ .* ((1:p) .- 1/2) ./ p |> collect
β = τ .* (1 .- (1:p) ./ p) |> collect
β[p] = τ / (4 * p)

times = range(0, 1, p+1);

In [ ]:
mf_problem = Problem(p, J)

In [ ]:
# initial spins
S = [[[1., 0., 0.] for _ in 1:N-1] for _ in 1:p+1]

# evolution with history
evolve!(S, mf_problem.local_fields, mf_problem.couplings, β, γ);

In [ ]:
# helper function to reformat the data
get_spin_data = n -> mapreduce(permutedims, vcat, [S[k][n] for k in 1:p+1]) |> transpose;

In [ ]:
# plot x, y, and z of all spins 
figure(figsize=((N - 1) * 2.2, 2))

for n in 1:N - 1
    subplot(1, N - 1, n)
    plot(times, get_spin_data(n)[1, 1:end])
    plot(times, get_spin_data(n)[2, 1:end])
    plot(times, get_spin_data(n)[3, 1:end])
    xlim(0, 1)
    ylim(-1, 1)
    xlabel("t/T")
    ylabel("n_" * string(n))
end
tight_layout()

In [ ]:
expectation(S[end], mf_problem.local_fields, mf_problem.couplings)

## Annealing

In [ ]:
T_anneal = 8.
p = 256
linear_schedule(t) = t / T_anneal
annealing_problem = QAOA.Problem(p, zeros(N), J);

In [ ]:
probs = anneal(annealing_problem, linear_schedule, T_anneal)